In [52]:
import pandas as pd 
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [53]:

# 创建映射字典
category_to_number = {'装': 1, '卸': -1, 'other': 0}

# 定义一个函数，用于将Category列的文本转换为对应数值
def convert_category(row):
    v=row['state'].strip()
    if v not in category_to_number:
        return 0
    return category_to_number[v]

df =pd.read_csv('./data/states.txt')
df['start'] =df.apply(lambda row:row['date']+' '+row['start_time'],axis=1)
df['end'] = df.apply(lambda row:row['date']+' '+row['end_time'],axis=1)
df[['start','end']]=df[['start','end']].apply(pd.to_datetime)
df['statev'] = df.apply(convert_category,axis=1)
df.to_csv('./data/states.csv')
df.head()

,date,start_time,end_time,state,start,end,statev
0,2023-12-17,10:44:49,10:48:59,装,2023-12-17 10:44:49,2023-12-17 10:48:59,1
1,2023-12-17,11:29:45,11:31:55,卸,2023-12-17 11:29:45,2023-12-17 11:31:55,-1
2,2023-12-17,12:23:56,12:29:44,装,2023-12-17 12:23:56,2023-12-17 12:29:44,1
3,2023-12-17,12:55:08,12:56:28,卸,2023-12-17 12:55:08,2023-12-17 12:56:28,-1
4,2023-12-17,13:45:02,13:45:42,装,2023-12-17 13:45:02,2023-12-17 13:45:42,1


In [ ]:
# 定义读取，处理数据函数
def fix_data(inputDF:pd.DataFrame)->pd.DataFrame:
    # 创建一个新的空列用于存储结果
    inputDF.insert(0,'state',0)
    
    # 对df1中的每一行遍历，并查找df2中符合条件的记录
    for index, row in inputDF.iterrows():
        condition = (df['start'] <= row['时间']) & (row['时间'] <= df['end'])
        match = df[condition]
        
        if not match.empty:
            # 如果找到了匹配项，则将df2的'state'赋值给df1的新列
            inputDF.at[index, 'state'] = match['statev'].values[0]
        else:
            # 如果没有找到匹配项，则保持原样（这里已经初始化为0）
            pass    
    return inputDF

def read_data(file_path:str)->pd.DataFrame:
    return fix_data( pd.read_excel(file_path,engine='openpyxl',parse_dates=['时间','轨迹时间']))

In [55]:




df17 =read_data("./data/data_17.xlsx")
df21 =read_data("./data/data_21.xlsx")
df22 =read_data("./data/data_22.xlsx")
df23 =read_data("./data/data_23.xlsx")
display(len(df17.query('state != 0')),len(df21.query('state != 0')),len(df22.query('state != 0')),len(df23.query('state != 0')))


557

340

845

1420